# Endpoint Creation

In [1]:
from sagemaker.huggingface.model import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/yravi/.config/sagemaker/config.yaml


In [2]:
role = "arn:aws:iam::011528263565:role/dev" # Past the ARN of the role created
endpoint_name = "fastapi-sentiment-classifier" # Name the endpoint

In [3]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sentiment-classification-fastapi/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Mention the Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # And Python version used
)

In [4]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
   endpoint_name= endpoint_name
)

# Print the endpoint name
print(f'Model deployed at endpoint: {predictor.endpoint_name}')

-----!Model deployed at endpoint: fastapi-sentiment-classifier


# Inference

In [8]:
import boto3
import json

In [6]:
endpoint_name = "fastapi-sentiment-classifier" 
boto_session = boto3.Session(region_name='ap-south-1')
sagemaker_runtime = boto_session.client('sagemaker-runtime')

In [9]:

input_data = {
   "inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}

payload = json.dumps(input_data)

# Make the prediction
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType='application/json'  # Set the content type to JSON
)

result = response['Body'].read().decode('utf-8')
print("Prediction result:", result)

Prediction result: [{"label":"NEU","score":0.993026614189148}]
